# Aplicando e Definindo o Modelo

Formatando o kernel e criando as camadas

In [165]:
from tensorflow.keras import models, layers, activations, initializers

model = models.Sequential([
    layers.Input(shape=(550, 550, 3)),   # definindo o modelo

    layers.Resizing(64, 64),           
    layers.Rescaling(1./255),  # salva pixels na mesma escala
    layers.RandomRotation((-0.2, 0.2)),

    # usado para identificar padrões em imagens
    layers.Conv2D(32, activation='relu', kernel_size=(3,3), strides=(1,1), padding='same'),     
    layers.MaxPooling2D(),   

    layers.Conv2D(64, activation='relu', kernel_size=(3,3), strides=(1,1), padding='same'),     
     # reduz o tamanho da imagem pegando apenas o valor máximo dentro de grids, evita o overfitting
    layers.MaxPooling2D(),          

    # transforma uma matriz em um vetor único 
    layers.Flatten(),     

     # camada com N neurônio, com ativação relu (para valores negativos, a saída é zero e positivos passam direto)
    # layers.Dense(
    #     128,       # unidade
    #     activations.softmax,   # função de ativação
    #     kernel_initializer=initializers.RandomNormal()), # inicialização dos pesos

    # elimina aleatóriamente 20% alguns neurônios durante a fase de treinamento
     

    # sigmoid: transforma qualquer número em um valor entre 0 e 1
    layers.Dense(
        128,
        activations.relu,
        kernel_initializer=initializers.RandomNormal()), 
    layers.Dropout(0.3), 

    # usado quando a rede precisa escolher uma classe. Pega um vetor de números e transforma em uma probabilidade que somam 1
    layers.Dense(
        2,    # a ultima camada tem que ser a quantidade de classes que o dataset possui
        activations.softmax,
        kernel_initializer=initializers.RandomNormal()), 

    #  Gelu : ReLU suave , probabilística
    # layers.Dense(
    #     64,
    #     activations.gelu,
    #     kernel_initializer=initializers.RandomNormal()),
])

# Otimização e Métricas
Calculando o erro (Adam - Mistura de momentum + adaptativo. É o mais usado, aprende mais rápido e estável)

In [166]:
from tensorflow.keras import optimizers, losses, metrics

lr = 0.0015

model.compile(
    optimizer = optimizers.Adam(
        learning_rate = lr
    ),
    loss = losses.SparseCategoricalCrossentropy(),  # mede a acurácia considerando rótulos inteiros (não one-hot)
    metrics = [ metrics.sparse_categorical_accuracy ]
)

# Lendo o Dataset

lendo as imagens de teste e de treino do dataset

In [167]:
from tensorflow.keras import utils

path = './Data'
batch_size = 100 

train = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    subset = 'training',
    validation_split = 0.1,
    seed = 1,
    image_size = (64, 64),
    batch_size = batch_size
)

test = utils.image_dataset_from_directory(
    directory = path,
    shuffle = True,
    subset = 'validation',
    validation_split = 0.1,
    seed = 1,
    image_size = (64, 64),
    batch_size = batch_size
)


Found 2000 files belonging to 2 classes.
Using 1800 files for training.
Found 2000 files belonging to 2 classes.
Using 200 files for validation.


# Treinando o Modelo

In [168]:
from tensorflow.keras import callbacks

patience = 5  # tolerância para evitar overfitting, controle de desempenho
epochs = 50  # épocas

model.fit(
    train,
    validation_data = test,
    epochs = epochs,
    verbose = True,   # verbose: Controla o nível de detalhamento do processo de treinamento:  0: Sem saída.    1: Barra de progresso.  2: Uma linha por época.

    callbacks = [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = True
        )
    ]
)

# loss = perda no treino (quanto ele erra, quanto menor, melhor)
# sparse = score (score do treino)
# val_loss = perda no teste
# val_sparse_accuracy = acurácia das imagens de teste

Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 97ms/step - loss: 0.7760 - sparse_categorical_accuracy: 0.4961 - val_loss: 0.6943 - val_sparse_categorical_accuracy: 0.4550
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - loss: 0.6928 - sparse_categorical_accuracy: 0.4994 - val_loss: 0.6899 - val_sparse_categorical_accuracy: 0.5750
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - loss: 0.6874 - sparse_categorical_accuracy: 0.5428 - val_loss: 0.6557 - val_sparse_categorical_accuracy: 0.6400
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - loss: 0.6717 - sparse_categorical_accuracy: 0.5928 - val_loss: 0.6615 - val_sparse_categorical_accuracy: 0.6150
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - loss: 0.6536 - sparse_categorical_accuracy: 0.6100 - val_loss: 0.6080 - val_sparse_categorical_accuracy: 0.6600
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - loss: 0.6432 - sparse_categorical_accuracy: 0.6206 - val_loss: 0.6312 - val_sparse_categorical_accuracy: 0.6200
Epoch 7/50